## 10. Collaboration

### 89 Consider `warnings` to Refactor and Migrate Usage

In [1]:
def print_distance(speed, duration):
    distance = speed * duration
    print(f'{distance} miles')

print_distance(5, 2.5)

12.5 miles


In [2]:
CONVERSIONS = {
    'mph': 1.60934 / 3600 * 1000,   # m/s
    'hours': 3600,                  # seconds
    'miles': 1.60934 * 1000,        # m
    'meters': 1,                    # m
    'm/s': 1,                       # m
    'seconds': 1,                   # s
}

def convert(value, units):
    rate = CONVERSIONS[units]
    return rate * value

def localize(value, units):
    rate = CONVERSIONS[units]
    return value / rate

def print_distance(speed, duration, *,
                   speed_units='mph',
                   time_units='hours',
                   distance_units='miles'):
    norm_speed = convert(speed, speed_units)
    norm_duration = convert(duration, time_units)
    norm_distance = norm_speed * norm_duration
    distance = localize(norm_distance, distance_units)
    print(f'{distance} {distance_units}')

In [3]:
print_distance(1000, 3,
               speed_units='meters',
               time_units='seconds')

1.8641182099494205 miles


In [4]:
import warnings

In [5]:
def print_distance(speed, duration, *,
                   speed_units=None,
                   time_units=None,
                   distance_units=None):
    if speed_units is None:
        warnings.warn('speed_units required', DeprecationWarning)
        speed_units = 'mph'

    if time_units is None:
        warnings.warn('time_units required', DeprecationWarning)
        time_units = 'hours'

    if distance_units is None:
        warnings.warn('distance_units required', DeprecationWarning)
        distance_units = 'miles'

    norm_speed = convert(speed, speed_units)
    norm_duration = convert(duration, time_units)
    norm_distance = norm_speed * norm_duration
    distance = localize(norm_distance, distance_units)
    print(f'{distance} {distance_units}')

In [6]:
import contextlib
import io

fake_stderr = io.StringIO()
with contextlib.redirect_stderr(fake_stderr):
    print_distance(1000, 3,
                   speed_units='meters',
                   time_units='seconds')

print(fake_stderr.getvalue())

1.8641182099494205 miles
<ipython-input-5-e1e3da18c016>:14: DeprecationWarning: distance_units required
  warnings.warn('distance_units required', DeprecationWarning)



In [7]:
def require(name, value, default):
    if value is not None:
        return value
    warnings.warn(
        f'{name} will be required soon, update your code',
        DeprecationWarning,
        stacklevel=3)
    return default

def print_distance(speed, duration, *,
                   speed_units=None,
                   time_units=None,
                   distance_units=None):
    speed_units = require('speed_units', speed_units, 'mph')
    time_units = require('time_units', time_units, 'hours')
    distance_units = require('distance_units', distance_units, 'miles')

    norm_speed = convert(speed, speed_units)
    norm_duration = convert(duration, time_units)
    norm_distance = norm_speed * norm_duration
    distance = localize(norm_distance, distance_units)
    print(f'{distance} {distance_units}')

In [8]:
import contextlib
import io

fake_stderr = io.StringIO()
with contextlib.redirect_stderr(fake_stderr):
    print_distance(1000, 3,
                   speed_units='meters',
                   time_units='seconds')

print(fake_stderr.getvalue())

1.8641182099494205 miles
<ipython-input-8-5969e2a95f81>:6: DeprecationWarning: distance_units will be required soon, update your code
  print_distance(1000, 3,



In [9]:
warnings.resetwarnings()

warnings.simplefilter('error')
try:
    warnings.warn('This usage is deprecated', DeprecationWarning)
except DeprecationWarning:
    print('DeprecationWarning caught')
    pass  # Expected
else:
    assert False

warnings.resetwarnings()

DeprecationWarning caught


```python
# ex09.py

import warnings

try:
    warnings.warn('This usage is deprecated', DeprecationWarning)
except DeprecationWarning:
    print('DeprecationWarning caught')
```

```shell
$ python item_89_ex_09.py
item_89_ex_09.py:8: DeprecationWarning: This usage is deprecated
  warnings.warn('This usage is deprecated', DeprecationWarning)

$ python -W error item_89_ex_09.py
DeprecationWarning caught

$ PYTHONWARNINGS=error python item_89_ex_09.py
DeprecationWarning caught
```

In [13]:
warnings.resetwarnings()

warnings.simplefilter('ignore')
warnings.warn('This will not be printed to stderr')

warnings.resetwarnings()

```shell
$ python
>>> import warnings
>>> warnings.warn('This usage is deprecated', DeprecationWarning)
<stdin>:1: DeprecationWarning: This usage is deprecated

$ python
>>> import warnings
>>> warnings.simplefilter('error')
>>> try:
...     warnings.warn('This usage is deprecated', DeprecationWarning)
... except DeprecationWarning:
...     print('DeprecationWarning caught')
... 
DeprecationWarning caught

$ python -W error
>>> import warnings
>>> warnings.warn('This usage is deprecated', DeprecationWarning)
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
DeprecationWarning: This usage is deprecated

$ PYTHONWARNINGS=error python
>>> import warnings
>>> try:
...     warnings.warn('This usage is deprecated', DeprecationWarning)
... except DeprecationWarning:
...     print('DeprecationWarning caught')
... 
DeprecationWarning caught

$ python
>>> import warnings
>>> warnings.simplefilter('ignore')
>>> warnings.warn('This usage is deprecated', DeprecationWarning)
```

In [14]:
import logging

fake_stderr = io.StringIO()
handler = logging.StreamHandler(fake_stderr)
formatter = logging.Formatter('%(asctime)-15s WARNING] %(message)s')
handler.setFormatter(formatter)

logging.captureWarnings(True)
logger = logging.getLogger('py.warnings')
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

warnings.resetwarnings()
warnings.simplefilter('default')
warnings.warn('This will go to the logs output')

print(fake_stderr.getvalue())

warnings.resetwarnings()

2021-05-08 19:39:58,940 WARNING] <ipython-input-14-a9e1ee1007ba>:15: UserWarning: This will go to the logs output
  warnings.warn('This will go to the logs output')




In [15]:
with warnings.catch_warnings(record=True) as found_warnings:
    found = require('my_arg', None, 'fake units')
    expected = 'fake units'
    assert found == expected

In [16]:
assert len(found_warnings) == 1
single_warning = found_warnings[0]
assert str(single_warning.message) == (
    'my_arg will be required soon, update your code')
assert single_warning.category == DeprecationWarning

> - `warnings` 모듈을 사용하면 여러분의 API를 호출하는 사용자들에게 앞으로 사용 금지될 사용법에 대해 알려줄 수 있다. 경고 메시지는 API 사용자들이 (API 변경으로 인해) 자신의 코드가 깨지기 전에 코드를 변경하도록 권장한다.
> - `-w error` 명령줄 인자를 파이썬 인터프리터에게 넘기면 경고를 오류로 높일 수 있다. 의존 관계에서 잠재적인 회귀 오류가 있는지 잡나내고 싶은 자동화 테스트에서 이런 기능이 특히 유용하다.
>   - `PYTHONWARNINGS` 환경 변수
> - 프로덕션 환경에서는 경고를 `logging` 모듈로 복제해 실행 시점에 기존 오류 보고 시스템이 경고를 잡아내게 할 수 있다.
> - 다운스트림 의존 관계에서 알맞은 때 경고가 발동되도록 코드가 생성하는 경고에 대해 데스트를 작성하면 유용하다.